# **distilbert-base-uncased基于PyTorch Trainer做微调训练**

# 1.安装环境

In [ ]:
%%writefile requirements.txt
datasets
transformers
evaluate
accelerate
scikit-learn
torch
torchvision
torchaudio

In [ ]:
pip install -r requirements.txt

# 2.加载数据

In [ ]:
from datasets import load_dataset

# 远程拉取
# dataset = load_dataset("yelp_review_full")

dataset = load_dataset(
    'parquet',
    data_files={
        'train': '../datasets/yelp_review_full/train-00000-of-00001.parquet',
        'test': '../datasets/yelp_review_full/test-00000-of-00001.parquet'
    }
)

dataset["train"][100]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../models/distilbert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

# 3.加载模型

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("../models/distilbert-base-uncased", num_labels=5)

# 4.准备训练

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    num_train_epochs=12,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    output_dir="trained_model"
)

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# 5.开始训练

In [ ]:
trainer.train()

# 6.测试训练结果

In [ ]:
# 文本特征提取
from transformers import DistilBertTokenizer, DistilBertModel
trained_tokenizer = DistilBertTokenizer.from_pretrained("../models/distilbert-base-uncased")
trained_model = DistilBertModel.from_pretrained("trained_model/checkpoint-72")
text = "Replace me by any text you'd like."
encoded_input = trained_tokenizer(text, return_tensors='pt')
outputs = trained_model(**encoded_input)
print(outputs)

In [ ]:
# 遮罩语言建模
from transformers import pipeline

from transformers import pipeline
unmasker = pipeline('fill-mask', model="../models/distilbert-base-uncased")
unmasker("Hello I'm a [MASK] model.")

In [ ]:
unmasker = pipeline('fill-mask', model='trained_model/checkpoint-72', tokenizer=tokenizer)
unmasker("Hello I'm a [MASK] model.")

# 7.清理环境

In [ ]:
del model
del trained_model
del tokenizer

del dataset
del tokenized_datasets
del small_train_dataset
del small_eval_dataset

In [ ]:
pip uninstall -y -r requirements.txt

In [ ]:
rm -rf requirements.txt